In [4]:
import tensorflow as tf
from tensorflow.contrib import rnn

In [40]:
def dilated_dynamic_rnn(cell, inputs, rates, sequence_length=None, initial_state=None, dtype=None, time_major=False, parallel_iterations=32, swap_memory=False):
    assert isinstance(cell, rnn.MultiRNNCell), 'param :cell has to be a MultiRNNCell instance.'
    if not time_major:
        inputs = tf.transpose(inputs, [1,0,2])
        
    max_time, batch_size, input_dim = tf.unstack(tf.shape(inputs))
    
    input_ta = tf.TensorArray(dtype=tf.float32, size=max_time)\
        .unstack(inputs)
        
    output_ta = tf.TensorArray(dtype=tf.float32, size=max_time)
    
    states_ta = [tf.TensorArray(dtype=tf.float32, size=rate) for rate in rates]
        
    if initial_state is not None:
        state = initial_state
    else:
        assert dtype is not None, '`dtype` must be provided when `initial_state`=None'
        state = cell.zero_state(batch_size, dtype)
    
    def _rnn_step():
        pass
    
    def _set_state(time, rate, states_ta):
        pass
    
    def _get_state(time, rate, states_ta):
        pass
        
    def _time_step(time, output_ta, state, states_ta):
        print(cell.state_size)
        call_cell = lambda: cell(input_ta.read(time), state)
        
        if sequence_length is not None:
            pass
        else:
            (output, new_state) = call_cell()
        
        output_ta.write(time, output)
            
        return (time + 1, output_ta, new_state, states_ta)
    
    time = tf.constant(0, dtype=tf.int32)
        
    _, output_final_ta, final_state, _ = tf.while_loop(
        cond = lambda time, *_: time < max_time,
        body = _time_step,
        loop_vars = (time, output_ta, state, states_ta),
        parallel_iterations = parallel_iterations,
        swap_memory = swap_memory
    )
    
    outputs = output_final_ta.stack()
    if not time_major:
        outputs = tf.transpose(outputs, [1,0,2])
        
    return outputs, final_state
        
with tf.Graph().as_default():
    inputs = tf.placeholder(tf.float32, [64, 12, 300])
    
    cell = rnn.MultiRNNCell([
        rnn.GRUCell(100),
        rnn.GRUCell(200)
    ])
    
    print(dilated_dynamic_rnn(cell, inputs, [1, 2], dtype=tf.float32))
    
        
    
    

(100, 200)
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
['File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main\n    "__main__", mod_spec)', 'File "/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code\n    exec(code, run_globals)', 'File "/Users/marekgalovic/.virtualenvs/data/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>\n    app.launch_new_instance()', 'File "/Users/marekgalovic/.virtualenvs/data/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance\n    app.start()', 'File "/Users/marekgalovic/.virtualenvs/data/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 474, in start\n    ioloop.IOLoop.instance().start()', 'File "/Us